In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver

#### Both link are the search results of 'covid' and restrict search results from Jan to April in 2020 sorted by oldest
NR: https://www.nationalreview.com/?s=covid&sp%5Bforce%5D=1&search-date=custom&search-date-from=01%2F01%2F2020&search-date-to=04%2F01%2F2020&orderby=asc&orderby=date&order=ASC

NYT: https://www.nytimes.com/search?dropmab=true&endDate=20200404&query=Covid&sort=oldest&startDate=20200101&types=article



## Get name and article links from National Review

In [33]:
def get_rightwing_aricle_links(searchresult,df):
    r = requests.get(searchresult)
    soup = BeautifulSoup(r.text, 'html.parser')
    for h4 in soup.find_all('h4')[:10]:
        df=df.append({'name': h4.a.contents[0],'link': h4.a['href']}, ignore_index=True)
    return df
def rsouce(page):
    return 'https://www.nationalreview.com/page/'+str(page)+'/?s=covid&sp%5Bforce%5D=1&search-post-type%5B0%5D=post&search-post-type%5B1%5D=magazine&search-post-type%5B2%5D=blog-post&search-date=custom&search-date-from=01%2F01%2F2020&search-date-to=04%2F01%2F2020&orderby=date&order=ASC'

In [36]:
# Get 1000 article title and link from NR
df=pd.DataFrame(columns=['name','link'])
page=1
while page<=100:
    df=get_rightwing_aricle_links(rsouce(page),df)
    page+=1

In [37]:
df.drop_duplicates()

,name,link
0,The Chinese Communist Party Struggles to Conta...,https://www.nationalreview.com/2020/02/coronav...
1,Escape from Wuhan,https://www.nationalreview.com/magazine/2020/0...
2,CDC Refused to Test First Possible Community-T...,https://www.nationalreview.com/news/cdc-refuse...
3,Coronavirus Chronicles,https://www.nationalreview.com/the-morning-jol...
4,Pence Announces White House Coordinator for Co...,https://www.nationalreview.com/news/pence-anno...
...,...,...
995,"Votes and conscience, &c.",https://www.nationalreview.com/2020/05/joe-bid...
996,Students: College Shouldn’t Be Your Default Now,https://www.nationalreview.com/corner/students...
997,COVID and Prisons,https://www.nationalreview.com/corner/covid-an...
998,"Science, Coronavirus, and Notre Dame",https://www.nationalreview.com/corner/science-...


In [38]:
df.to_csv('right_wing_textonly.csv',index=False)

## Get name and article links from New York Times

In [101]:
def get_article_from_nyt(total):
    driver = webdriver.Chrome()
    url = 'https://www.nytimes.com/search?dropmab=true&endDate=20200404&query=Covid&sort=oldest&startDate=20200101&types=article'
    driver.get(url)
    i=0
    while i<total:
        driver.find_element_by_xpath('//button[normalize-space()="Show More"]').click()
        i+=1
    return driver.page_source

In [102]:
soup = BeautifulSoup(get_article_from_nyt(100), 'html.parser')

In [103]:
prefix='https://www.nytimes.com'
df=pd.DataFrame(columns=['name','link'])
for h4 in soup.find_all('h4')[1:]:
    df=df.append({'name': h4.contents[0],'link': prefix+h4.parent['href']}, ignore_index=True)

In [104]:
df.drop_duplicates()

,name,link
0,China Identifies New Virus Causing Pneumoniali...,https://www.nytimes.com/2020/01/08/health/chin...
1,China Reports First Death From New Virus,https://www.nytimes.com/2020/01/10/world/asia/...
2,Japan and Thailand Confirm New Cases of Chines...,https://www.nytimes.com/2020/01/15/world/asia/...
3,Three U.S. Airports to Check Passengers for a ...,https://www.nytimes.com/2020/01/17/health/chin...
4,Deadly Mystery Virus Reported in 2 New Chinese...,https://www.nytimes.com/2020/01/18/world/asia/...
...,...,...
935,Coronavirus Outbreak Tests World’s Dependence ...,https://www.nytimes.com/2020/01/29/business/ch...
936,How the Coronavirus Could Hurt Apple and Starb...,https://www.nytimes.com/2020/01/29/business/de...
937,Coronavirus in New York: Lunar New Year Events...,https://www.nytimes.com/2020/01/29/nyregion/co...
938,Mask Hoarders May Raise Risk of a Coronavirus ...,https://www.nytimes.com/2020/01/29/health/coro...


In [105]:
#df.to_csv('left_wing.csv',index=False)

## Get articles from links

In [40]:
#read link file
right_link=pd.read_csv('right_wing_textonly.csv')
left_link=pd.read_csv('left_wing.csv')

#### Get left wing articles

In [51]:
df=pd.DataFrame(columns=['article'])
for link in left_link['link']:
    txt=''
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    for p in soup.find_all('p',attrs={"class":"css-158dogj evys1bk0"}):
        if p.contents:
            txt=str(p.contents[0])+txt
    df=df.append({'article': txt}, ignore_index=True)

6060
2770
5037
4961
4597
5900
7412
6368
8443
6845
8716
6231
4072
4225
18008
1785
4811
4030
5703
5880
8716
6231
4072
4225
18008
1785
4811
4030
5703
5880
8716
6231
4072
4225
18008
1785
4811
4030
5703
5880
8716
6231
4072
4225
18008
1785
4811
4030
5703
5880
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2990
5315
3856
5712
5271
2888
8360
3273
4586
17873
2

In [54]:
df['label']=1
df['title']=left_link['name']

In [56]:
#df.to_csv('left_articles.csv',index=False)

#### Get Right wing articles

In [73]:
df=pd.DataFrame(columns=['article'])
driver = webdriver.Chrome()
for link in right_link['link'][528:]:
    txt=''
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for p in soup.find_all('p'):
        if p.contents:
            txt=str(p.contents[0])+txt
    print(len(txt))
    df=df.append({'article': txt}, ignore_index=True)

4417
2310
2212
2308
7855
2079
1611
879
6333
0
0
0
264
519
1822
3886
3346
5308
3978
276
7215
0
51611
2412
608
773
6995
24
378
5851
8213
824
1684
398
963
2239
726
3244
5418
2927
0
0
910
7789
0
8362
2087
1892
3006
1680
2240
3329
1298
746
276
276
276
4994
24
1397
1406
6241
1947
2656
7731
2269
1527
1585
490
644
0
4932
0
3349
2155
603
6060
1305
0
0
351
2552
0
1401
1920
1988
194
2157
0
0
0
0
417
0
674
9425
2417
7682
4811
2163
2310
1994
2225
1326
2770
1287
3515
608
225
230
1986
8474
572
40481
2864
5731
2018
6056
2980
1477
767
0
0
0
0
0
0
7987
4134
3411
5454
1510
2772
572
276
3502
0
0
276
7435
2314
0
2928
1694
3867
3595
1960
589
503
812
519
1724
6328
1537
377
1982
3017
5648
0
0
4657
4151
4111
595
0
2711
753
517
7273
276
4353
508
758
6208
1432
659
212
0
1091
1160
1403
883
4324
1366
1327
1397
24
0
2061
2489
7999
1698
276
3931
2544
601
1282
1576
1728
1022
276
552
4150
4061
276
1340
8059
1598
3304
715
5426
0
0
2745
2391
4836
800
276
47605
593
3110
5779
4617
1986
0
1670
3849
1644
3510
3150
1799
2646

In [ ]:
df['label']=0
df['title']=right_link['name']

In [78]:
#Drop empty string in the dataframe
df.replace("", float("NaN"), inplace=True)
df=df.dropna()

In [87]:
#df.to_csv('right_articles.csv',index=False)